# 🗓️ Week 07: Putting it all together: from web scraping to initial data cleaning

In this lecture, we revisit the core concepts of web scraping by compiling a list of the last instances of [UK general elections from Wikipedia](https://en.wikipedia.org/wiki/List_of_United_Kingdom_general_elections).

The case study covers the following topics:

- Finding CSS/XPath selectors on a page
- Writing functions for web scraping tasks
- List comprehensions for data extraction
- Using pd.apply() for data manipulation

The repository will be created from scratch during the lecture, providing a hands-on approach to Git commands and web scraping techniques. This case study aims to reinforce the concepts learned and help students apply them to their W08 assignment.


In [9]:
import io
import requests

import pandas as pd
import numpy as np

from scrapy.selector import Selector
from urllib.parse import urljoin

### Part 1: Pages about UK general elections

In [32]:
base_url = "https://en.wikipedia.org/wiki/List_of_United_Kingdom_general_elections"

response = requests.get(base_url)
sel = Selector(text=response.text)

response.status_code

200

In [48]:
def box_info(box):
    common_url = "https://en.wikipedia.org"
    text = box.xpath(".//text()").get()
    href = box.xpath(".//@href").get()

    return {
        "year": box.xpath(".//text()").get(),
        "url": urljoin(common_url, href)
    }

big_box = sel.xpath("//td[contains(@class, 'navbox-list')]")[0]

boxes = big_box.xpath(".//a")
pages = [box_info(box) for box in boxes]

df_links = pd.DataFrame(pages)

df_links.head(10)

,year,url
0,1801,https://en.wikipedia.org/wiki/First_Parliament...
1,1802,https://en.wikipedia.org/wiki/1802_United_King...
2,1806,https://en.wikipedia.org/wiki/1806_United_King...
3,1807,https://en.wikipedia.org/wiki/1807_United_King...
4,1812,https://en.wikipedia.org/wiki/1812_United_King...
5,1818,https://en.wikipedia.org/wiki/1818_United_King...
6,1820,https://en.wikipedia.org/wiki/1820_United_King...
7,1826,https://en.wikipedia.org/wiki/1826_United_King...
8,1830,https://en.wikipedia.org/wiki/1830_United_King...
9,1831,https://en.wikipedia.org/wiki/1831_United_King...


In [50]:
len(df_links)

59

Intermediate step to get the page with breakdown of results (subpage within 2019 page):

In [64]:
last_elections_page = df_links[df_links["year"] == "2019"]
last_url = elections_page["url"]

response = requests.get(last_url)
sel = Selector(text=response.text)

#Select the <a> that contains the text "Results of the " in its title
results_breakdown = sel.xpath("//a[contains(@title, 'Results of the ')]").css("::attr(href)").extract_first()
results_breakdown = urljoin("https://en.wikipedia.org", results_breakdown)

results_breakdown

'https://en.wikipedia.org/wiki/Results_of_the_2019_United_Kingdom_general_election'

Extract the tables from the 2019 subpage

In [70]:
response = requests.get(results_breakdown)
sel = Selector(text=response.text)

sel.css("table.wikitable")

#Convert the CSS selector to XPath
table = sel.xpath("(//table[contains(@class, 'wikitable')])[1]")

print(table.extract_first())

<table class="wikitable sortable" style="text-align:right; font-size:95%">

<tbody><tr>
<th rowspan="3" scope="col" style="text-align:left;"><a href="/wiki/United_Kingdom_Parliament_constituencies" class="mw-redirect" title="United Kingdom Parliament constituencies">Constituency</a>
</th>
<th rowspan="3" scope="col"><a href="/wiki/Counties_of_the_United_Kingdom" title="Counties of the United Kingdom">County</a>
</th>
<th rowspan="3" scope="col"><a href="/wiki/Regions_of_England" title="Regions of England">Region</a>
</th>
<th colspan="2" rowspan="3" scope="col">2017<br>result
</th>
<th colspan="5">2019 winning party
</th>
<th rowspan="3" scope="col">Turnout<sup id="cite_ref-spoilts_2-0" class="reference"><a href="#cite_note-spoilts-2">[a]</a></sup>
</th>
<th colspan="7" scope="col">Votes
</th></tr>
<tr>
<th colspan="2" rowspan="2">Party
</th>
<th rowspan="2" scope="col">Votes
</th>
<th rowspan="2" scope="col">Share
</th>
<th rowspan="2" scope="col">Majority
</th>
<th scope="col"><a hre

In [71]:
#MultiIndex and repeated rows at the bottom
pd.read_html(table.extract_first())[0]

/var/folders/_s/n53dl30x6x90zrn97fjh8_6m0000gn/T/ipykernel_26837/1552295266.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(table.extract_first())[0]


Constituency                        County  \
                     Constituency                        County   
                     Constituency                        County   
0                             NaN                           NaN   
1                       Aldershot                           HAM   
2             Aldridge-Brownhills                           WMD   
3        Altrincham and Sale West                           GTM   
4                    Amber Valley                           DBY   
..                            ...                           ...   
537  Total for all constituencies  Total for all constituencies   
538  Total for all constituencies  Total for all constituencies   
539  Total for all constituencies  Total for all constituencies   
540  Total for all constituencies  Total for all constituencies   
541  Total for all constituencies  Total for all constituencies   

                           Region                   2017 result  \
                           Region                   2017 result   
                           Region                   2017 result   
0                             NaN                           NaN   
1                              SE                           NaN   
2                              WM                           NaN   
3                              NW                           NaN   
4                              EM                           NaN   
..                            ...                           ...   
537  Total for all constituencies  Total for all constituencies   
538  Total for all constituencies  Total for all constituencies   
539  Total for all constituencies  Total for all constituencies   
540  Total for all constituencies  Total for all constituencies   
541  Total for all constituencies  Total for all constituencies   

                                             2019 winning party  \
                                                          Party   
                    2017 result.1                         Party   
0                             NaN                           NaN   
1                             Con                           NaN   
2                             Con                           NaN   
3                             Con                           NaN   
4                             Con                           NaN   
..                            ...                           ...   
537  Total for all constituencies  Total for all constituencies   
538  Total for all constituencies  Total for all constituencies   
539  Total for all constituencies  Total for all constituencies   
540  Total for all constituencies  Total for all constituencies   
541  Total for all constituencies  Total for all constituencies   

                                                                 \
                                                          Votes   
                          Party.1                         Votes   
0                             NaN                           NaN   
1                             Con                         27980   
2                             Con                         27850   
3                             Con                         26311   
4                             Con                         29096   
..                            ...                           ...   
537  Total for all constituencies  Total for all constituencies   
538  Total for all constituencies  Total for all constituencies   
539  Total for all constituencies  Total for all constituencies   
540  Total for all constituencies  Total for all constituencies   
541  Total for all constituencies  Total for all constituencies   

                                                                Turnout[a]  \
                            Share                      Majority Turnout[a]   
                            Share                      Majority Turnout[a]   
0                             N

In [81]:
#Scraping the table directly without pandas
columns = table.xpath("./tbody/tr[th and not(contains(@class, 'sortbottom'))]").extract()
len(columns)

4

In [80]:
rows = table.xpath("./tbody/tr[not(th)]")
len(rows)

533

In [82]:
rows[0].xpath("./td/text()").extract()

['\n',
 '\n',
 '\n',
 '27,980\n',
 '58.4%\n',
 '16,698\n',
 '66.0%\n',
 '27,980\n',
 '11,282\n',
 '6,920\n',
 '1,750\n',
 '\n',
 '\n',
 '47,932\n']